<a href="https://colab.research.google.com/github/jdklee/PatentClassification_CNN/blob/master/stacked_CNN_with_BERT_tokenizer_on_Patent_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2
!pip install tensorflow
!pip install tensorflow_hub
!pip install gcsfs
!pip install transformers
# !pip install google.colab



     |████████████████████████████████| 40kB 1.8MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.6-cp36-none-any.whl size=30318 sha256=ed849b0eaa78fb29fe046c96ec3d70fa4cedb8c25181e60cae84d6301134afa6
  Stored in directory: /root/.cache/pip/wheels/07/a0/b4/75b0601ebaa41e517a797fe9cea119c789664c8408f8a74ae9
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=f9d7ec278d8b285505419146bc7df244342e6f594ddeffccb616cec18820a941
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=94918ae3dce08a39541dfd042da98b736feb4f7bb5c634edd986d99447d01b20
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 92kB 3.4MB/s 
     |██████████████████████████████

In [2]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np
import pandas as pd
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
!pip install bert
import bert
import gc
import math
import random

import numpy as np
!pip install transformers
from transformers import *
# auth.authenticate_user()
# print('Authenticated')


  Created wheel for bert: filename=bert-2.2.0-cp36-none-any.whl size=3756 sha256=f8b9be1e6e14cd52fb280aed83ba5272e311c304731012c6041c58efb34aaac7
  Stored in directory: /root/.cache/pip/wheels/fe/71/b7/941459453bd38e5d97a8c886361dee19325e9933c9cf88ad46
  Created wheel for erlastic: filename=erlastic-2.0.0-cp36-none-any.whl size=6789 sha256=9d0203bdb9b99a993e6e89bdd21ef72c1eed31999ea1c8345abd778a0f018657
  Stored in directory: /root/.cache/pip/wheels/02/62/46/93c713a5f061aeeb4f16eb6bf5ee798816e6ddda70faa78e69
Successfully built bert erlastic


In [3]:
! pip install keras


In [4]:
# # something like this to use TPU

# log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# TPU_ADDRESS = " node-1"
# TPU_ZONE = "us-central1-f"
# USE_TPU =True
# NUM_TPU_CORES = 8
# try:
#   device_name = os.environ['COLAB_TPU_ADDR']
#   TPU_ADDRESS = 'grpc://' + device_name
#   print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#   print('TPU not found')

#  tf.config.experimental_connect_to_host(TPU_ADDRESS)
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver) 

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))
# strategy = tf.distribute.experimental.TPUStrategy(resolver)


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)



INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.38.244.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.244.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
#Index(['patent_id', 'fwrdCitations_5', 'publication_date', 'text'], dtype='object')
"""LOAD DATA"""

PATH='gs://patents-research/patent_research/data_frwdcorrect.tsv'


def loadData(gsPath):
    return pd.read_csv(gsPath, sep = "\t")

def generateLable(dataset):
    # convert to datetime
    dataset['publication_date'] = pd.to_datetime(dataset['publication_date'], errors="coerce",format="%Y-%m-%d")
    dataset = dataset.sort_values('publication_date', ascending = False)

    #drop if date is NaN - only one 1082-03-15
    dataset = dataset[dataset.publication_date.isnull() == False]
    
    # calculate the top 5% by publication date - give it label 1
    top1_perc =  dataset.groupby(dataset.publication_date.dt.year)["fwrdCitations_5"].transform(lambda x: x.quantile(.99)) #change this quantile to .99
    dataset["label"] = dataset["fwrdCitations_5"] >= top1_perc
    
    # calculate top 5% by publication date - give it label 2
    top5_perc = dataset.groupby(dataset.publication_date.dt.year)["fwrdCitations_5"].transform(lambda x: x.quantile(.95))
    dataset["label"] = np.where(np.logical_and(dataset["fwrdCitations_5"] >= top5_perc, dataset["label"]==0), 2, dataset["label"])
    
    return dataset

def saveToGloud(path,data):
    '''Saves to gcloud so we dont have to do this long ass step every time'''
    with file_io.FileIO(path, mode='w') as f:
        pickle.dump(data,f)


def readTFRecord(path):
    with file_io.FileIO(path, mode='rb') as f:
        return pickle.load(f)
with strategy.scope():
  df=loadData(PATH)
  df=generateLable(df)
  print("df loaded")


FileNotFoundError: ignored

In [6]:
small_df=df.iloc[0:500]

NameError: ignored

In [ ]:
"""CREATE dataset from dataframe"""
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
def tokenize_dataframe(df):
  def tokenize_tweets(tweets):
      return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets))
  df["token"]=df["text"].apply(lambda x: tokenize_tweets(x))
  return df


def sort_dataframe(df):
  s = df.token.str.len().sort_values().index
  return df.reindex(s)

def get_token_label_tuples(df):
  return list(zip(*map(df.get, ['token', 'label'])  ))

# df=df.drop(["patent_id", 'fwrdCitations_5', 'publication_date'], axis=1)

# df=tokenize_dataframe(df)
# print("tokenized df")

# df=df.drop(["text"], axis=1)

# df=sort_dataframe(df)
# print("sort df")

# tuples=get_token_label_tuples(df)
# print("created tuple")
# del df
# gc.collect()

small_df=tokenize_dataframe(small_df)
small_df=sort_dataframe(small_df)

tuples=get_token_label_tuples(small_df)

In [ ]:
### HYPERPARAMS CHANGE 
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 3 ###CHANGE THIS IF MULTICLASS

DROPOUT_RATE = 0.4

NB_EPOCHS = 5

BATCH_SIZE=8 ##CHANGE THIS FOR LARGER DATASET
TOTAL_BATCHES = math.ceil(len(tuples) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10

In [ ]:
def generate_batched_dataset(tuples, batch_size):
  processed_dataset = tf.data.Dataset.from_generator(lambda: tuples, output_types=(tf.int32, tf.int32))
  batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()) )
  return batched_dataset

def get_train_test_data(batched_dataset, batch_size):

  test_data = batched_dataset.take(TEST_BATCHES)
  train_data = batched_dataset.skip(TEST_BATCHES)

batched_dataset=generate_batched_dataset(tuples,BATCH_SIZE)

dataset = tf.data.Dataset.from_tensor_slices((small_df["token"],small_df["label"]))


print("generated batched dataset")
# del(tuples)
gc.collect()
#processed_dataset = tf.data.Dataset.from_generator(lambda: tuples, output_types=(tf.int32, tf.int32))
# processed_dataset = tf.data.Dataset.from_tensor_slices((features, target))

#BATCH_SIZE = 8
#batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))


# TOTAL_BATCHES = math.ceil(len(tuples) / BATCH_SIZE)
# TEST_BATCHES = TOTAL_BATCHES // 10
print(TOTAL_BATCHES,TEST_BATCHES)
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

ValueError: ignored

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy", "Precision", "Recall", "AUC", "FalseNegatives", "FalsePositives", "TrueNegatives","TruePositives"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])#,tf.keras.metrics.Precision(class_id=1),
                                #tf.keras.metrics.Precision(class_id=2),tf.keras.metrics.Precision(class_id=0)])

<SkipDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [ ]:

# Based on -> https://github.com/tensorflow/tpu/blob/master/models/experimental/resnet50_keras/resnet50_tf2.py
with strategy.scope():

  text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE) 
  #text_model.compile()
  if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy", "Precision", "Recall", "AUC", "FalseNegatives", "FalsePositives", "TrueNegatives","TruePositives"])
  else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])#,tf.keras.metrics.Precision(class_id=1),
                                #tf.keras.metrics.Precision(class_id=2),tf.keras.metrics.Precision(class_id=0)])
text_model.fit(train_data, epochs=NB_EPOCHS) #callbacks=[tensorboard_callback])


Epoch 1/5
57/57 [==============================] - 1s 10ms/step - loss: 0.3019 - sparse_categorical_accuracy: 0.9403
Epoch 2/5
57/57 [==============================] - 0s 8ms/step - loss: 0.2106 - sparse_categorical_accuracy: 0.9469
Epoch 3/5
57/57 [==============================] - 1s 9ms/step - loss: 0.1505 - sparse_categorical_accuracy: 0.9469
Epoch 4/5
57/57 [==============================] - 0s 8ms/step - loss: 0.0677 - sparse_categorical_accuracy: 0.9735
Epoch 5/5
57/57 [==============================] - 0s 9ms/step - loss: 0.0236 - sparse_categorical_accuracy: 0.9978


In [ ]:

# log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# text_model.fit(train_data, epochs=NB_EPOCHS, callbacks=[tensorboard_callback])
# # 

In [ ]:
# results = text_model.evaluate(test_data)
# print(results)

In [ ]:
###EVALUATE (PREDICT ON TEST DATA)

def get_test_x_y(test_data):
  test_x=[]
  test_y=[]
  for x in test_data:
    test_x.append(x[0])
    test_y.append(x[1])
  return test_x, test_y
  
def get_TN_TP_FN_FP(model, test_data):
  test_x,test_y=get_test_x_y(test_data)
  predictions=np.array([model.predict(x).argmax(axis=1) for x in test_x]).flatten()
  true_labels=np.array(test_y).flatten()
  tn=tf.keras.metrics.TrueNegatives()
  tn.update_state(true_labels,predictions)
  TN=tn.result()
  tp=tf.keras.metrics.TruePositives()
  tp.update_state(true_labels,predictions)
  TP=tp.result()
  fn=tf.keras.metrics.FalseNegatives()
  fn.update_state(true_labels,predictions)
  FN=fn.result()
  fp=tf.keras.metrics.FalsePositives()
  fp.update_state(true_labels,predictions)
  FP=fp.result()
  return TN.numpy(),TP.numpy(),FN.numpy(),FP.numpy()

TN,TP,FN,FP=get_TN_TP_FN_FP(text_model, test_data)

def get_recall_specificity_precision_accuracy(TN,TP,FN,FP):
  # Sensitivity, hit rate, recall, or true positive rate
  if TP+FN==0:
    TPR=0.0
  else:
    TPR = TP/(TP+FN)
  # Specificity or true negative rate
  if TN+FP==0:
    TNR=0.0
  else:
    TNR = TN/(TN+FP) 
  # Precision or positive predictive value
  if TP+FP==0:
    PPV=0.0
  else:
    PPV = TP/(TP+FP)
  # Overall accuracy
  ACC = (TP+TN)/(TP+FP+FN+TN)
  return TPR,TNR,PPV,ACC

Recall,Specificity,Precision,Accuracy = get_recall_specificity_precision_accuracy(TN,TP,FN,FP)
print(" Recall: ", Recall,"\n", "Specificity: ", Specificity,"\n", "Precision: ", Precision, "\n", "Accuracy: ",Accuracy,"\n")

Results=pd.DataFrame([Recall, Specificity, Precision, Accuracy, TN, TP, FN, FP])
Results.to_csv("Results.csv")


In [ ]:
def get_cm(model, test_data):
  test_x,test_y=get_test_x_y(test_data)
  predictions=np.array([model.predict(x).argmax(axis=1) for x in test_x]).flatten()
  true_labels=np.array(test_y).flatten()
  cm=tf.math.confusion_matrix(true_labels, predictions)
  return cm

cm=get_cm(text_model, test_data)

print(cm)

In [ ]:
x=('[9046, 16742, 3752, 4733, 579, 760, 579, 2532, 962, 3901, 579, 1542, 16548, 119, 17805, 1374, 450, 962, 864, 579, 1542, 16548, 119, 17805, 1374, 962, 3901, 579, 1542, 16548, 119, 1374, 4924, 1241, 3495, 30138, 6729, 18941, 2901, 7667, 4352, 2855, 30132, 4710, 2986, 4582, 30131, 7898, 2901, 5576, 30131, 7898, 3404, 10711, 8707, 17828, 1146, 962, 864, 579, 1542, 16548, 119, 1374, 6696, 4844, 15960, 422, 4089, 422, 2111, 3405, 422, 137, 21012, 624, 106, 6696, 4844, 15960, 434, 4535, 145, 158, 546, 19749, 182, 30130, 165, 151, 422, 125, 422, 6090, 234, 19077, 25017, 7085, 422, 182, 30132, 137, 182, 30138, 220, 151, 234, 106, 12020, 27830, 23498, 583, 422, 182, 30140, 147, 182, 30142, 535, 220, 106, 12020, 27830, 6696, 4844, 23498, 583, 422, 106, 165, 106, 3612, 21172, 23498, 583, 422, 137, 231, 30130, 165, 244, 422, 158, 234, 170, 205, 106, 4089, 2865, 263, 111, 6696, 4844, 15960, 552, 195, 28189, 190, 2826, 1506, 2462, 21504, 2550, 422, 1506, 15872, 422, 1397, 1209, 483, 137, 11015, 137, 165, 2700, 188, 130, 9288, 4089, 121, 4535, 586, 106, 2111, 3405, 205, 962, 3901, 579, 4332, 30130, 1374, 158, 205, 106, 6696, 4844, 15960, 2773, 111, 1196, 4535, 145, 158, 546, 862, 19749, 182, 962, 864, 579, 4332, 30130, 1374]', 0)
temp=[]
tokens=x[0].strip("[]").split(", ")
for i in tokens:
  temp.append(int(i))

temp

[9046,
 16742,
 3752,
 4733,
 579,
 760,
 579,
 2532,
 962,
 3901,
 579,
 1542,
 16548,
 119,
 17805,
 1374,
 450,
 962,
 864,
 579,
 1542,
 16548,
 119,
 17805,
 1374,
 962,
 3901,
 579,
 1542,
 16548,
 119,
 1374,
 4924,
 1241,
 3495,
 30138,
 6729,
 18941,
 2901,
 7667,
 4352,
 2855,
 30132,
 4710,
 2986,
 4582,
 30131,
 7898,
 2901,
 5576,
 30131,
 7898,
 3404,
 10711,
 8707,
 17828,
 1146,
 962,
 864,
 579,
 1542,
 16548,
 119,
 1374,
 6696,
 4844,
 15960,
 422,
 4089,
 422,
 2111,
 3405,
 422,
 137,
 21012,
 624,
 106,
 6696,
 4844,
 15960,
 434,
 4535,
 145,
 158,
 546,
 19749,
 182,
 30130,
 165,
 151,
 422,
 125,
 422,
 6090,
 234,
 19077,
 25017,
 7085,
 422,
 182,
 30132,
 137,
 182,
 30138,
 220,
 151,
 234,
 106,
 12020,
 27830,
 23498,
 583,
 422,
 182,
 30140,
 147,
 182,
 30142,
 535,
 220,
 106,
 12020,
 27830,
 6696,
 4844,
 23498,
 583,
 422,
 106,
 165,
 106,
 3612,
 21172,
 23498,
 583,
 422,
 137,
 231,
 30130,
 165,
 244,
 422,
 158,
 234,
 170,
 205,
 106,
 4089

In [ ]:
i="[9094]"
i=i.strip("[]")
#i=i.strip("]")
i
int(i)

9094

In [ ]:
def reset_df(df):
  def strip_stuff(string_list):
    temp=[]
    tokens=string_list.strip("[]").split(", ")
    for i in tokens:
      temp.append(int(i))
    return np.array(temp)
  df["token"]=df["token"].apply(lambda x: strip_stuff(x))
  return df

ValueError: ignored

In [ ]:
x[0]

'[9046, 16742, 3752, 4733, 579, 760, 579, 2532, 962, 3901, 579, 1542, 16548, 119, 17805, 1374, 450, 962, 864, 579, 1542, 16548, 119, 17805, 1374, 962, 3901, 579, 1542, 16548, 119, 1374, 4924, 1241, 3495, 30138, 6729, 18941, 2901, 7667, 4352, 2855, 30132, 4710, 2986, 4582, 30131, 7898, 2901, 5576, 30131, 7898, 3404, 10711, 8707, 17828, 1146, 962, 864, 579, 1542, 16548, 119, 1374, 6696, 4844, 15960, 422, 4089, 422, 2111, 3405, 422, 137, 21012, 624, 106, 6696, 4844, 15960, 434, 4535, 145, 158, 546, 19749, 182, 30130, 165, 151, 422, 125, 422, 6090, 234, 19077, 25017, 7085, 422, 182, 30132, 137, 182, 30138, 220, 151, 234, 106, 12020, 27830, 23498, 583, 422, 182, 30140, 147, 182, 30142, 535, 220, 106, 12020, 27830, 6696, 4844, 23498, 583, 422, 106, 165, 106, 3612, 21172, 23498, 583, 422, 137, 231, 30130, 165, 244, 422, 158, 234, 170, 205, 106, 4089, 2865, 263, 111, 6696, 4844, 15960, 552, 195, 28189, 190, 2826, 1506, 2462, 21504, 2550, 422, 1506, 15872, 422, 1397, 1209, 483, 137, 11015, 137,

In [ ]:
import numpy as np
np.array([0,2,2,0,1,0])/2

array([0. , 1. , 1. , 0. , 0.5, 0. ])

In [ ]:
import tensorflow as tf
import numpy as np
predictions = tf.convert_to_tensor(value=np.array([1,2,3,4,5]), dtype=tf.int32)
true_labels=tf.convert_to_tensor(value=np.array([1,2,3,4,5]),dtype=tf.int32)

predictions

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 2, 3, 4, 5], dtype=int32)>

In [ ]:
true_labels

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 2, 3, 4, 5], dtype=int32)>

In [ ]:

import os
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import numpy as np
import pandas as pd
import csv

import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import gc
import math
import random
import keras
import numpy as np

from transformers import *

"""LOAD DATA"""




def loadData(gsPath):
  return pd.read_csv(gsPath, sep = "\t")

def generateLable(dataset):
# convert to datetime
  dataset['publication_date'] = pd.to_datetime(dataset['publication_date'], errors="coerce",format="%Y-%m-%d")
  dataset = dataset.sort_values('publication_date', ascending = False)

  #drop if date is NaN - only one 1082-03-15
  dataset = dataset[dataset.publication_date.isnull() == False]
    
  # calculate the top 5% by publication date - give it label 1
  top1_perc =  dataset.groupby(dataset.publication_date.dt.year)["fwrdCitations_5"].transform(lambda x: x.quantile(.99)) #change this quantile to .99
  dataset["label"] = dataset["fwrdCitations_5"] >= top1_perc
    
  # calculate top 5% by publication date - give it label 2
  top5_perc = dataset.groupby(dataset.publication_date.dt.year)["fwrdCitations_5"].transform(lambda x: x.quantile(.95))
  dataset["label"] = np.where(np.logical_and(dataset["fwrdCitations_5"] >= top5_perc, dataset["label"]==0), 2, dataset["label"])
    
  return dataset

def saveToGloud(path,data):
  '''Saves to gcloud so we dont have to do this long ass step every time'''
  with file_io.FileIO(path, mode='w') as f:
    pickle.dump(data,f)


def readTFRecord(path):
  with file_io.FileIO(path, mode='rb') as f:
    return pickle.load(f)


#small_df=df.iloc[0:500]
"""CREATE dataset from dataframe"""

def tokenize_dataframe(df,tokenizer):

  def tokenize_tweets(tweets):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets))
  df["token"]=df["text"].apply(lambda x: tokenize_tweets(x))

  return df

def sort_dataframe(df):
  s = df.token.str.len().sort_values().index
  return df.reindex(s)

def reset_df(df):
  def strip_stuff(string_list):
    temp=[]
    tokens=string_list.strip("[]").split(", ")
    for i in tokens:
      temp.append(int(i))
    return np.array(temp)
  df["token"]=df["token"].apply(lambda x: strip_stuff(x))
  return df

def get_token_label_tuples(df):
  return list(zip(*map(df.get, ['token', 'label'])))




def generate_batched_dataset(tuples, batch_size):
  processed_dataset = tf.data.Dataset.from_generator(lambda: tuples, output_types=(tf.int32, tf.int32))
  batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()) )
  return batched_dataset

def get_train_test_data(batched_dataset, batch_size):

  test_data = batched_dataset.take(TEST_BATCHES)
  train_data = batched_dataset.skip(TEST_BATCHES)



class TEXT_MODEL(tf.keras.Model):
  def __init__(self, vocabulary_size, embedding_dimensions=128, cnn_filters=50, dnn_units=512, model_output_classes=3, dropout_rate=0.1, training=False, name="text_model"):
    super(TEXT_MODEL, self).__init__(name=name)
        
    self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
    self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
    self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
    self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
    self.pool = layers.GlobalMaxPool1D()
        
    self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
    self.dropout = layers.Dropout(rate=dropout_rate)
    if model_output_classes == 2:
      self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
    else:
      self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
  def call(self, inputs, training):
    l = self.embedding(inputs)
    l_1 = self.cnn_layer1(l) 
    l_1 = self.pool(l_1) 
    l_2 = self.cnn_layer2(l) 
    l_2 = self.pool(l_2)
    l_3 = self.cnn_layer3(l)
    l_3 = self.pool(l_3) 
    concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
    concatenated = self.dense_1(concatenated)
    concatenated = self.dropout(concatenated, training)
    model_output = self.last_dense(concatenated)
    return model_output

def build_model(vocabulary_size, embedding_dimensions, cnn_filters, dnn_units, model_output_classes, dropout_rate):
  return TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)


# text_model = build_model(vocabulary_size=VOCAB_LENGTH,
#                         embedding_dimensions=EMB_DIM,
#                         cnn_filters=CNN_FILTERS,
#                         dnn_units=DNN_UNITS,
#                         model_output_classes=OUTPUT_CLASSES,
#                         dropout_rate=DROPOUT_RATE)
def compile_model(output_classes,text_model):
  if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy", "Precision", "Recall", "AUC", "FalseNegatives", "FalsePositives", "TrueNegatives","TruePositives"])
  else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                   metrics=["sparse_categorical_accuracy"])





###EVALUATE (PREDICT ON TEST DATA)

def get_test_x_y(test_data):
  test_x=[]
  test_y=[]
  for x in test_data:
    test_x.append(x[0])
    test_y.append(x[1])
  return test_x, test_y
  
# def get_TN_TP_FN_FP(model, test_data):
#   test_x,test_y=get_test_x_y(test_data)
#   predictions=np.array([model.predict(x).argmax(axis=1) for x in test_x]).flatten()
#   predictions=predictions/(OUTPUT_CLASSES-1)
#   true_labels=np.array(test_y).flatten()
#   true_labels=true_labels/(OUTPUT_CLASSES-1)
#   #print("pred: ",predictions, "labels: ",true_labels)
#   tn=tf.keras.metrics.TrueNegatives()
#   tn.update_state(true_labels,predictions)
#   TN=tn.result()
#   tp=tf.keras.metrics.TruePositives()
#   tp.update_state(true_labels,predictions)
#   TP=tp.result()
#   fn=tf.keras.metrics.FalseNegatives()
#   fn.update_state(true_labels,predictions)
#   FN=fn.result()
#   fp=tf.keras.metrics.FalsePositives()
#   fp.update_state(true_labels,predictions)
#   FP=fp.result()
#   return TN.numpy(),TP.numpy(),FN.numpy(),FP.numpy()



# def get_recall_specificity_precision_accuracy(TN,TP,FN,FP):
#   # Sensitivity, hit rate, recall, or true positive rate
#   if TP+FN==0:
#     TPR=0.0
#   else:
#     TPR = TP/(TP+FN)
#   # Specificity or true negative rate
#   if TN+FP==0:
#     TNR=0.0
#   else:
#     TNR = TN/(TN+FP) 
#   # Precision or positive predictive value
#   if TP+FP==0:
#     PPV=0.0
#   else:
#     PPV = TP/(TP+FP)
#   # Overall accuracy
#   ACC = (TP+TN)/(TP+FP+FN+TN)
#   return TPR,TNR,PPV,ACC

def get_cm(model, test_data):


  test_x,test_y=get_test_x_y(test_data)
  predictions=np.asarray([model.predict(x).argmax(axis=1) for x in test_x]).flatten()
  true_labels=np.asarray(test_y).flatten()
  predictions = tf.convert_to_tensor(predictions, dtype=tf.int32)
  true_labels=tf.convert_to_tensor(true_labels,dtype=tf.int32)
  cm=tf.math.confusion_matrix(true_labels, predictions)
  return cm

def get_train_cm(model, train_data):
    def get_test_x_y_2(test_data):
      test_x=[]
      test_y=[]
      for x in test_data:
        if len(x[0])==BATCH_SIZE:
          test_x.append(x[0])
        if len(x[1])==BATCH_SIZE:
          test_y.append(x[1])
      return test_x, test_y
    test_x,test_y=get_test_x_y_2(train_data)
    predictions=np.asarray([model.predict(x).argmax(axis=1) for x in test_x]).flatten()
    true_labels=np.asarray(test_y).flatten()
    predictions = tf.convert_to_tensor(predictions, dtype=tf.int32)
    true_labels=tf.convert_to_tensor(true_labels,dtype=tf.int32)
    cm=tf.math.confusion_matrix(true_labels, predictions)
    return cm


if __name__ == '__main__':
  import pandas as pd
  import os
  from sklearn.model_selection import train_test_split
  import pandas as pd
  import tensorflow as tf
  import tensorflow_hub as hub
  from datetime import datetime
  import numpy as np
  import pandas as pd
  import csv
  import tensorflow as tf

  import tensorflow_hub as hub
  from tensorflow.keras import layers
  import bert
  import gc
  import math
  import random

  import numpy as np

  from transformers import *
  
  # resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-1')
  # tf.config.experimental_connect_to_cluster(resolver)
  # # This is the TPU initialization code that has to be at the beginning.
  # tf.tpu.experimental.initialize_tpu_system(resolver)
  # print("All devices: ", tf.config.list_logical_devices('TPU'))
  # strategy = tf.distribute.experimental.TPUStrategy(resolver)

  ###LOAD DF
  # PATH='gs://patents-research/patent_research/data_frwdcorrect.tsv'
  # df=loadData(PATH)
  # df=generateLable(df)
  # print("df loaded")
  # small_df=df.iloc[0:500]
  # ###PROESS DF###
  tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

  # df=df.drop(["patent_id", 'fwrdCitations_5', 'publication_date'], axis=1)
  # df=tokenize_dataframe(df,tokenizer)
  # print("tokenized df")

  # df=df.drop(["text"], axis=1)
  # df.to_csv("tokenized_df.csv")
  print("Loading TOKENIZED DATASET")
  # df=pd.read_csv("gs://dataset_patent/tokenized_df.csv")

  #df=pd.read_csv("tokenized_df.csv")
  

  small_df=df.iloc[0:500]
  print("LOADED DATASET")
  
  # columns=list(small_df.columns)
  print("columns: ",df.columns)
  small_df=small_df.drop("Unnamed: 0", axis=1)
  

  small_df=reset_df(small_df)
  print("reset df")


  small_df=sort_dataframe(small_df)
  print("sort df")
  tuples=get_token_label_tuples(small_df)
  print("created tuple")
  #del small_df
  gc.collect()
  # small_df=tokenize_dataframe(small_df, tokenizer)
  # print("tokenized df")
  # small_df=sort_dataframe(small_df)
  # print("sort df")
  # tuples=get_token_label_tuples(small_df)
  # print("created tuple")

  ###Hyper PARAMETERS######
  VOCAB_LENGTH = len(tokenizer.vocab)
  EMB_DIM = 200
  CNN_FILTERS = 100
  DNN_UNITS = 256
  OUTPUT_CLASSES = 3 ###CHANGE THIS IF MULTICLASS
  DROPOUT_RATE = 0.4
  NB_EPOCHS = 8
  BATCH_SIZE=8 ##CHANGE THIS FOR LARGER DATASET
  TOTAL_BATCHES = math.ceil(len(tuples) / BATCH_SIZE)
  TEST_BATCHES = TOTAL_BATCHES // 10
  VALIDATION_BATCHES=TOTAL_BATCHES // 10
  
  #### GENERATE DATASET
  batched_dataset=generate_batched_dataset(tuples,BATCH_SIZE)

  print("generated batched dataset")

  print(TOTAL_BATCHES,TEST_BATCHES)

  batched_dataset.shuffle(TOTAL_BATCHES)
  test_data = batched_dataset.take(TEST_BATCHES)
  validation_data=batched_dataset.take(VALIDATION_BATCHES)
  train_data = batched_dataset.skip(TEST_BATCHES+VALIDATION_BATCHES)
  del tuples
  gc.collect()


  ##BUILD MODEL
  text_model = build_model(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)
  
  compile_model(OUTPUT_CLASSES, text_model)

  callbacks = [EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1)]
  text_model.fit(train_data,
          epochs=NB_EPOCHS,
          batch_size=BATCH_SIZE,
          callbacks=callbacks,
          validation_data=validation_data)
  # text_model.fit(train_data, epochs=NB_EPOCHS ,validation_data=validation_data)
    
  ### EVAL MODEL

  # TN,TP,FN,FP=get_TN_TP_FN_FP(text_model, test_data)
  # Recall,Specificity,Precision,Accuracy = get_recall_specificity_precision_accuracy(TN,TP,FN,FP)
  cm=get_cm(text_model, test_data)

  # print(" Recall: ", Recall,"\n", "Specificity: ", Specificity,"\n", "Precision: ", Precision, "\n", "Accuracy: ",Accuracy,"\n",
  #       "TrueNegatives: ",TN,"\n","TruePositives: ",TP,"\n","FalseNegatives: ",FN,"\n","FalsePositives: ",FP,"\n",)
  print("test cm",cm)

  with open("test_confusion_matrix.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(cm)
  train_cm=get_train_cm(text_model, train_data)
  print("train cm", train_cm)
  with open("train_confusion_matrix.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(train_cm)

  # Results=pd.DataFrame([Recall, Specificity, Precision, Accuracy, TN, TP, FN, FP])
  # Results.to_csv("Results.csv")



Loading TOKENIZED DATASET
LOADED DATASET
columns:  Index(['Unnamed: 0', 'label', 'token'], dtype='object')
reset df
sort df
created tuple
generated batched dataset
63 6
Epoch 1/8
51/51 [==============================] - 3s 63ms/step - loss: 0.3020 - sparse_categorical_accuracy: 0.9455 - val_loss: 0.3103 - val_sparse_categorical_accuracy: 0.9375
Epoch 2/8
51/51 [==============================] - 3s 61ms/step - loss: 0.2113 - sparse_categorical_accuracy: 0.9480 - val_loss: 0.3259 - val_sparse_categorical_accuracy: 0.9375
Epoch 3/8
51/51 [==============================] - 3s 60ms/step - loss: 0.1675 - sparse_categorical_accuracy: 0.9480 - val_loss: 0.3091 - val_sparse_categorical_accuracy: 0.9375
Epoch 00003: early stopping
test cm tf.Tensor(
[[45  0  0]
 [ 1  0  0]
 [ 2  0  0]], shape=(3, 3), dtype=int32)
train cm tf.Tensor(
[[379   0   0]
 [  1   0   0]
 [ 20   0   0]], shape=(3, 3), dtype=int32)


In [ ]:
  
  callbacks = [EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=5e-3,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=4,
        verbose=1)]
  text_model.fit(train_data,
          epochs=NB_EPOCHS,
          batch_size=BATCH_SIZE,
          callbacks=callbacks,
          validation_data=validation_data)
  # text_model.fit(train_data, epochs=NB_EPOCHS ,validation_data=validation_data)
    
  ### EVAL MODEL

  TN,TP,FN,FP=get_TN_TP_FN_FP(text_model, test_data)
  Recall,Specificity,Precision,Accuracy = get_recall_specificity_precision_accuracy(TN,TP,FN,FP)
  cm=get_cm(text_model, test_data)

  print(" Recall: ", Recall,"\n", "Specificity: ", Specificity,"\n", "Precision: ", Precision, "\n", "Accuracy: ",Accuracy,"\n",
        "TrueNegatives: ",TN,"\n","TruePositives: ",TP,"\n","FalseNegatives: ",FN,"\n","FalsePositives: ",FP,"\n",)
  print("test cm",cm)
  train_cm=get_cm(text_model, train_data)
  print("train cm", cm)

  Results=pd.DataFrame([Recall, Specificity, Precision, Accuracy, TN, TP, FN, FP])
  Results.to_csv("Results.csv")

Epoch 1/8
51/51 [==============================] - 3s 62ms/step - loss: 0.0956 - sparse_categorical_accuracy: 0.9579 - val_loss: 0.3144 - val_sparse_categorical_accuracy: 0.9375
Epoch 2/8
51/51 [==============================] - 3s 62ms/step - loss: 0.0382 - sparse_categorical_accuracy: 0.9950 - val_loss: 0.3459 - val_sparse_categorical_accuracy: 0.9375
Epoch 3/8
51/51 [==============================] - 3s 61ms/step - loss: 0.0180 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.4714 - val_sparse_categorical_accuracy: 0.9375
Epoch 4/8
51/51 [==============================] - 3s 61ms/step - loss: 0.0127 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.4010 - val_sparse_categorical_accuracy: 0.9375
Epoch 5/8
51/51 [==============================] - 3s 61ms/step - loss: 0.0136 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.5089 - val_sparse_categorical_accuracy: 0.9375
Epoch 00005: early stopping
 Recall:  0.0 
 Specificity:  1.0 
 Precision:  0.0 
 Accuracy:  0.9375 
 TrueNega

ValueError: ignored

In [ ]:
train_cm=get_cm(text_model, train_data)

ValueError: ignored

In [ ]:
  test_x,test_y=get_test_x_y(train_data)
  predictions=np.array([text_model.predict(x).argmax(axis=1) for x in test_x]).flatten()
  true_labels=np.array(test_y).flatten()
  
  # predictions = tf.convert_to_tensor(predictions, dtype=tf.int32)
  # true_labels=tf.convert_to_tensor(true_labels,dtype=tf.int32)
  # cm=tf.math.confusion_matrix(true_labels, predictions)

In [ ]:
temp=tf.tensor()
for x in predictions:
   temp.append(tf.convert_to_tensor(x, dtype=tf.int32))


AttributeError: ignored

In [ ]:
predictions.shape

(51,)

In [ ]:
  test_x,test_y=get_test_x_y(train_data)
  predictions=np.asarray([text_model.predict(x).argmax(axis=1) for x in test_x]).flatten()
  true_labels=np.asarray(test_y).flatten()
  # predictions = tf.convert_to_tensor(predictions, dtype=tf.int32)
  # true_labels=tf.convert_to_tensor(true_labels,dtype=tf.int32)
  # cm=tf.math.confusion_matrix(true_labels, predictions)

In [ ]:
predictions, true_labels

(array([array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]),
        array([0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
train_data.shape

AttributeError: ignored

In [ ]:
10000//1.6

6249.0